<a href="https://colab.research.google.com/github/LCaravaggio/DeepLearning_ITBA/blob/main/CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c cifar100-dl-itba-2020q2

Mounted at /content/drive
  0% 0.00/456 [00:00<?, ?B/s]
100% 456/456 [00:00<00:00, 829kB/s]
  0% 0.00/1.30k [00:00<?, ?B/s]
100% 1.30k/1.30k [00:00<00:00, 3.17MB/s]
  0% 0.00/391k [00:00<?, ?B/s]
100% 391k/391k [00:00<00:00, 52.6MB/s]
  0% 0.00/391k [00:00<?, ?B/s]
100% 391k/391k [00:00<00:00, 46.0MB/s]
 96% 129M/134M [00:01<00:00, 56.0MB/s]
100% 134M/134M [00:01<00:00, 71.1MB/s]
 86% 23.0M/26.8M [00:00<00:00, 83.9MB/s]
100% 26.8M/26.8M [00:00<00:00, 105MB/s] 


In [2]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [3]:
import numpy as np

x_test=np.load("x_test.npy")
x_train=np.load("x_train.npy")
y_train_coarse=np.load("y_train_coarse.npy")
y_train_fine=np.load("y_train_fine.npy")

#for file in os.listdir():
#    if file.endswith('.npy'):
#      np.load(file)

In [4]:
def norm (x): 
  return (x-np.mean(x))/np.std(x)

x_test=norm(x_test)
x_train=norm(x_train)
y_train_coarse=norm(y_train_coarse)
y_train_fine=norm(y_train_fine)

In [5]:
if not os.path.exists('mlp_helper.py'):
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/mlp_helper.py
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/fnn_helper.py
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/MNIST_helper.py
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/draw_nn.py

--2021-06-04 02:03:17--  https://github.com/lab-ml-itba/MLP-2019/raw/master/mlp_helper.py
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lab-ml-itba/MLP-2019/master/mlp_helper.py [following]
--2021-06-04 02:03:17--  https://raw.githubusercontent.com/lab-ml-itba/MLP-2019/master/mlp_helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13863 (14K) [text/plain]
Saving to: ‘mlp_helper.py’

mlp_helper.py       100%[===================>]  13.54K  --.-KB/s    in 0.001s  

2021-06-04 02:03:17 (20.2 MB/s) - ‘mlp_helper.py’ saved [13863/13863]

--2021-06-04 02:03:17--  https://github.com/lab-ml-itba/

In [6]:
import os.path
import tensorflow.keras
import numpy as np
from matplotlib import pyplot as plt
from fnn_helper import PlotLosses
from MNIST_helper import plot_numbers, visualize_input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.layers.advanced_activations import LeakyReLU
from fnn_helper import PlotLosses
#from keras_contrib.callbacks import CyclicLR
from matplotlib import pyplot as plt
from mlp_helper import plot_confusion_matrix
#import draw_nn

In [ ]:
np.unique(y_train_fine)

array([-1.71481604, -1.68017329, -1.64553055, -1.6108878 , -1.57624505,
       -1.5416023 , -1.50695955, -1.4723168 , -1.43767406, -1.40303131,
       -1.36838856, -1.33374581, -1.29910306, -1.26446031, -1.22981757,
       -1.19517482, -1.16053207, -1.12588932, -1.09124657, -1.05660382,
       -1.02196108, -0.98731833, -0.95267558, -0.91803283, -0.88339008,
       -0.84874733, -0.81410459, -0.77946184, -0.74481909, -0.71017634,
       -0.67553359, -0.64089084, -0.6062481 , -0.57160535, -0.5369626 ,
       -0.50231985, -0.4676771 , -0.43303435, -0.39839161, -0.36374886,
       -0.32910611, -0.29446336, -0.25982061, -0.22517786, -0.19053512,
       -0.15589237, -0.12124962, -0.08660687, -0.05196412, -0.01732137,
        0.01732137,  0.05196412,  0.08660687,  0.12124962,  0.15589237,
        0.19053512,  0.22517786,  0.25982061,  0.29446336,  0.32910611,
        0.36374886,  0.39839161,  0.43303435,  0.4676771 ,  0.50231985,
        0.5369626 ,  0.57160535,  0.6062481 ,  0.64089084,  0.67

In [13]:
def get_model(activations=['tanh', 'sigmoid']):
    output_size = 100
    model_plane = Sequential()
    model_plane.add(Flatten(input_shape=x_train.shape[1:]))
    model_plane.add(Dense(1000, activation=activations[0],  name='middle1'))
    #model_plane.add(Dropout(0.5))
    model_plane.add(Dense(750, activation=activations[1],  name='middle2'))
    #model_plane.add(Dropout(0.5))
    #model_plane.add(LeakyReLU())
    #model_plane.add(LeakyReLU())
    model_plane.add(Dense(output_size, activation='softmax',  name='salida'))
    return model_plane

In [14]:
model_plane = get_model(['tanh', 'sigmoid'])
model_plane.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 3072)              0         
_________________________________________________________________
middle1 (Dense)              (None, 1000)              3073000   
_________________________________________________________________
middle2 (Dense)              (None, 750)               750750    
_________________________________________________________________
salida (Dense)               (None, 100)               75100     
Total params: 3,898,850
Trainable params: 3,898,850
Non-trainable params: 0
_________________________________________________________________


In [23]:
batch_size=1

In [27]:
model_plane = get_model()

checkpointer = ModelCheckpoint(filepath='two-layer-2d.mnist.hdf5', verbose=1, save_best_only=True, mode='max', monitor='val_accuracy')
plot_losses = PlotLosses(plot_interval=1, evaluate_interval=None, x_val=x_train, y_val_categorical=y_train_fine)
# clr = CyclicLR(1e-6, 1e-4, 8*int(len(x_train)/batch_size))

In [28]:
optim = Adam(learning_rate=0.0000001) # 1e-06
model_plane.compile(loss="mse", optimizer=optim, metrics=['accuracy'])

In [ ]:
model_plane.fit(x_train, 
          y_train_fine ,
          epochs=10, batch_size=batch_size, 
          #verbose=1, 
          validation_data=(x_train, y_train_fine), 
          callbacks=[checkpointer],
         )

Epoch 1/10


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f91a148a680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


    1/50000 [..............................] - ETA: 857:44:11 - loss: 0.2095 - accuracy: 0.0000e+00

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f91a148a680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


  568/50000 [..............................] - ETA: 15:01 - loss: 1.0040 - accuracy: 0.0000e+00

In [ ]:
model_plane.evaluate(x_train, y_train_fine)

In [ ]:
from keras import optimizers

In [ ]:
def get_two_layer_model(input_shape, output_size, hidden_units= 10, lr=0.1, decay=0.0):
    model = Sequential()
    sgd = optimizers.SGD(lr=lr, decay=decay)
    model.add(Dense(hidden_units,input_dim=input_shape,  activation='sigmoid', ))
    model.add(Dense(output_size, 
                    activation='sigmoid', 
                    kernel_initializer='zeros', 
                    name='Salida'
                   ))
    model.compile(loss = 'binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
two_layer_model = get_two_layer_model(x_train.shape[1], 1, lr=1)
two_layer_model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                330       
_________________________________________________________________
Salida (Dense)               (None, 1)                 11        
Total params: 341
Trainable params: 341
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
plot_losses = PlotLosses(plot_interval=50, evaluate_interval=None, x_val=x_val, y_val_categorical=y_val_categorical)
checkpointer = ModelCheckpoint(filepath='two_layer_model.hdf5', verbose=0, save_best_only=True)
two_layer_model = get_two_layer_model(X_train.shape[1], 1, lr=2)
two_layer_model.fit(x_train, 
          y_train_fine, batch_size = 25,
          epochs=3500, 
          verbose=0, 
          validation_data=(X_test, y_test), 
          callbacks=[plot_losses, checkpointer],
         )

NameError: ignored